# Dependencies

In [1]:
!pip install lightning
!pip install torchtext.data

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement torchtext.data (from versions: none)
ERROR: No matching distribution found for torchtext.data


In [2]:
# file management
import sys
import shutil
import urllib
import tarfile
from pathlib import Path
import zipfile

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

# TASK 1: Corpus

Download the corpus

In [3]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)

        
def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def extract_dataset(download_path: Path, extract_path: Path):
    print("Extracting dataset... (it may take a while...)")
    with zipfile.ZipFile(download_path, 'r') as zip_file:
        zip_file.extractall(extract_path)

    print("Extraction completed!")

In [4]:
url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

print(f"Current work directory: {Path.cwd()}")

dataset_folder = Path.cwd().joinpath("Datasets")

if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_zip_path.exists():
    download_dataset(dataset_zip_path, url)

if not dataset_path.exists():
    extract_dataset(dataset_zip_path, dataset_folder)
  

Current work directory: /home/elements72/unibo/Vanno/NLP/Assignments/NLP/A1


Encoding the corpus into a pandas.DataFrame object

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

In [5]:
dataframe_rows = []  # list for DataFrame rows
for i, file_path in enumerate(dataset_path.iterdir()):
    if file_path.is_file(): # split corpus documents in the tree categories: train, validation, tests
        if 1 <= i + 1 <= 100:
            split = 'train'
        elif 101 <= i + 1 <= 150:
            split = 'validation'
        else:
            split = 'test'

        print(f"Processing {file_path.name}...")
        with file_path.open(mode='r', encoding='utf-8') as text_file: # read corpus lines
                lines = text_file.readlines()

        if len(lines) > 0:
        # split the first line based on tabs
            fields = lines[0].strip().split('\t')
        if len(fields) >= 2:
                text = fields[0]  # store the first field as 'text'
                POS = fields[1]   # store the second field as 'POS'

                dataframe_row = {  #build DataFrame rows
                    "text": text,
                    "POS": POS,
                    "split": split
                }

                dataframe_rows.append(dataframe_row) #append rows

# corpus DataFrame
corpus_df = pd.DataFrame(dataframe_rows) 

Processing wsj_0033.dp...
Processing wsj_0068.dp...
Processing wsj_0076.dp...
Processing wsj_0177.dp...
Processing wsj_0082.dp...
Processing wsj_0129.dp...
Processing wsj_0078.dp...
Processing wsj_0149.dp...
Processing wsj_0057.dp...
Processing wsj_0162.dp...
Processing wsj_0174.dp...
Processing wsj_0123.dp...
Processing wsj_0020.dp...
Processing wsj_0195.dp...
Processing wsj_0161.dp...
Processing wsj_0001.dp...
Processing wsj_0193.dp...
Processing wsj_0039.dp...
Processing wsj_0036.dp...
Processing wsj_0074.dp...
Processing wsj_0059.dp...
Processing wsj_0062.dp...
Processing wsj_0048.dp...
Processing wsj_0109.dp...
Processing wsj_0188.dp...
Processing wsj_0198.dp...
Processing wsj_0029.dp...
Processing wsj_0124.dp...
Processing wsj_0079.dp...
Processing wsj_0167.dp...
Processing wsj_0120.dp...
Processing wsj_0140.dp...
Processing wsj_0131.dp...
Processing wsj_0081.dp...
Processing wsj_0031.dp...
Processing wsj_0191.dp...
Processing wsj_0019.dp...
Processing wsj_0058.dp...
Processing w

Data inspection

In [6]:
corpus_df.head(10)

,text,POS,split
0,Japan,NNP,train
1,GOODY,NNP,train
2,Lancaster,NNP,train
3,Komatsu,NNP,train
4,Criticism,NNP,train
5,Lewis,NNP,train
6,Mitsui,NNP,train
7,New,NNP,train
8,LSI,NNP,train
9,A,DT,train


In [7]:
print("Dataframe structure:")
print(corpus_df)
print()

print("Total rows %d" % (len(corpus_df)))
print()

Dataframe structure:
          text  POS  split
0        Japan  NNP  train
1        GOODY  NNP  train
2    Lancaster  NNP  train
3      Komatsu  NNP  train
4    Criticism  NNP  train
..         ...  ...    ...
194          @   IN   test
195     PAPERS  NNS   test
196   Heritage  NNP   test
197   Genetics  NNP   test
198       When  WRB   test

[199 rows x 3 columns]

Total rows 199



## TASK 2: Text encoding

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

Encode text into numerical format

In [11]:
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe

In [24]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(embedding_dimension: int = 50):
    emb_model = GloVe(name="6B", dim=300)
    return emb_model

In [27]:
tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch

load_embedding_model()

.vector_cache/glove.6B.zip:  14%|█▍        | 124M/862M [10:41<1:03:26, 194kB/s]    


ContentTooShortError: <urlopen error retrieval incomplete: got only 124305149 out of 862182613 bytes>